# XRoration (XGate) - QKeras ML Models

## Configuration

Pulse and fidelity computation are computation intensive (slow).

In [1]:
PULSE_ENABLED = False
FIDELITY_ENABLED = True

Parameters are stored in a shared script.

In [2]:
from parameters import *

Enable seeding for reproducibility of the training.

## Library

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

import sys
sys.path.append('..')

from utils.helpers import *

# Disable some console warnings
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2024-04-20 15:41:17.958343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Check GPU

tf 2.11 seems to be too greedy with memory and needs to be limited if you plan to have multiple scripts with active python kernels

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5928961867548052656
xla_global_id: -1
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9804320277159547267
physical_device_desc: "device: XLA_CPU device"
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7768309760
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8627887128877027264
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11941867608357572273
physical_device_desc: "device: XLA_GPU device"
xla_global_id: -1
]


## Dataset

### Load

In [6]:
x_train = np.load('./data/xrotation/{}/x_train.npy'.format(CSV_CONFIG), allow_pickle=True)
x_val = np.load('./data/xrotation/{}/x_val.npy'.format(CSV_CONFIG), allow_pickle=True)
x_test = np.load('./data/xrotation/{}/x_test.npy'.format(CSV_CONFIG), allow_pickle=True)
y_train = np.load('./data/xrotation/{}/y_train.npy'.format(CSV_CONFIG), allow_pickle=True)
y_val = np.load('./data/xrotation/{}/y_val.npy'.format(CSV_CONFIG), allow_pickle=True)
y_test = np.load('./data/xrotation/{}/y_test.npy'.format(CSV_CONFIG), allow_pickle=True)

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

In [8]:
N_ANGLES = x_train.shape[1]
N_PARAMS = y_train.shape[1]

In [9]:
print('---------------------------------------')
print('- Pulse parameters (y) #', N_PARAMS) # ASSUMING 1 BETA VALUE (ANGLE ALONG X AXIS)
print('- Gate parameters  (x) #', N_ANGLES) # ASSUMING 1 BETA VALUE (ANGLES ALONG X AXIS)
print('---------------------------------------')

---------------------------------------
- Pulse parameters (y) # 5
- Gate parameters  (x) # 1
---------------------------------------


### Evaluation

<span style="color:red">ATTENTION: Measuring fidelity on the entire train, validation, and test sets could be time consuming. Use `limit` parameter in case.</span>

## MLP

### Load Keras Model

In [10]:
from tensorflow.keras.layers import Layer

In [11]:
best_model_file = PREFIX + '/best_keras_model.h5'
def sine_activation(x):
    return tf.math.sin(x)
model = tf.keras.models.load_model(best_model_file, custom_objects={'sine_activation': sine_activation})
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',
    metrics=['mean_squared_error'])

In [12]:
model_id_qkeras = MODEL_ID_PREFIX + get_basic_id(model)
print(model_id_qkeras)

smallMLP_1x4x5


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc0 (Dense)                 (None, 4)                 8         
                                                                 
 relu0 (Activation)          (None, 4)                 0         
                                                                 
 fc1 (Dense)                 (None, 5)                 25        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


### QAT/QKeras

#### Create model

A classical multi-layer perceptron: 3 (inputs), 8 hidden layers w/ 128 neurons, and 20 (outputs).

<span style="background-color:orange">This may be oversized for the final hardware implementation, but it is a starting point. The bit-precision is likely oversized too. We can use AutoQKeras or manually tune the hyper-parameters.</span>

In [14]:
import qkeras
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

Remember that `ap_fixed<16, 8, true>` is `quantized_bits(bits=16,integer=7)`.

In [15]:
# Build quantized model
W=21
I=5
QN=1
A=1

qmodel = tf.keras.models.Sequential()
#qmodel.add(Input(shape=(1,), name='input1'))
qmodel.add(QDense(NEURONS_PER_LAYER[0],
                  input_shape=(1,),
                  name='fc0',
                  kernel_quantizer=quantized_bits(bits=W,integer=I,alpha=A,qnoise_factor=QN),
                  bias_quantizer=quantized_bits(bits=W,integer=I,alpha=A,qnoise_factor=QN),
                  kernel_initializer='lecun_uniform'))
qmodel.add(QActivation(activation=quantized_relu(bits=W,integer=I,qnoise_factor=QN),
                       name='relu0'))

for i, n in enumerate(NEURONS_PER_LAYER[1:]):
    qmodel.add(QDense(n,
                  name='fc{}'.format(i+1),
                  kernel_quantizer=quantized_bits(bits=W,integer=I,alpha=A,qnoise_factor=QN),
                  bias_quantizer=quantized_bits(bits=W,integer=I,alpha=A,qnoise_factor=QN),
                  kernel_initializer='lecun_uniform'))
    qmodel.add(QActivation(activation=quantized_relu(bits=W,integer=I,qnoise_factor=QN),
                       name='relu{}'.format(i+1)))
qmodel.add(QDense(N_PARAMS,
                  name='fc{}'.format(len(NEURONS_PER_LAYER)),
                  kernel_quantizer=quantized_bits(bits=W,integer=I,alpha=A,qnoise_factor=QN),
                  bias_quantizer=quantized_bits(bits=W,integer=I,alpha=A,qnoise_factor=QN),
                  kernel_initializer='lecun_uniform'))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [16]:
qmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc0 (QDense)                (None, 4)                 8         
                                                                 
 relu0 (QActivation)         (None, 4)                 0         
                                                                 
 fc1 (QDense)                (None, 5)                 25        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


##### Training

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [18]:
best_model_file_qkeras = PREFIX + '/best_qkeras_model.h5'
last_model_file_qkeras = PREFIX + '/last_qkeras_model.h5'

Enable training (`train_and_save`) to overwrite the model file.

In [19]:
train_and_save = True

Set weights from the Keras model into the QKeras model

In [20]:
qmodel.set_weights(model.get_weights())
qmodel.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',
    metrics=['mean_squared_error'])
qmodel.save(best_model_file_qkeras)

##### We need not train for now if we simply try porting the weights from the Keras model

We use Adam optimizer, minimize the Mean Squared Error, and early stop.

Load the saved best model and use it from now on.

##### Evaluation

Although we may plot and print many metrics, we focus only on **Mean Squared Error (MSE).**

Plot training history. (Not needed if no retraining of QKeras model is done)

Print metrics.

In [21]:
qmse = qmodel.evaluate(x_test, y_test)[0]
qmsle, qmape = None, None

26/26 [==============================] - 0s 1ms/step - loss: 1.2713e-05 - mean_squared_error: 1.2713e-05


In [22]:
y_qkeras = qmodel.predict(x_test)

26/26 [==============================] - 0s 776us/step


In [23]:
mse = model.evaluate(x_test, y_test)[0]
msle, mape = None, None

26/26 [==============================] - 0s 854us/step - loss: 1.2692e-05 - mean_squared_error: 1.2692e-05


In [24]:
y_keras = model.predict(x_test)

26/26 [==============================] - 0s 602us/step


In [25]:
import random
hash_id = random.getrandbits(32)

if FIDELITY_ENABLED:
    xgate_fidelity_qkeras = get_xgate_fidelity(x_test,
                                               recover_original_structure(y_qkeras),
                                               config_template=CONFIG_TEMPLATE_JSON,
                                               pulse_data_path='/tmp/xgate_{:x}.csv'.format(hash_id),
                                               output_objf_path='/tmp/xgate_fidelity_{:x}.csv'.format(hash_id))
else:
    xgate_fidelity_qkeras = None

In [26]:
with pd.option_context('display.float_format', '{:0.12f}'.format):
    data = pd.DataFrame([["QKeras", qmse, qmsle, qmape, xgate_fidelity_qkeras]], columns=["", "MSE", "MSLE", "MAPE", "Fidelity"])
    display(data)

,,MSE,MSLE,MAPE,Fidelity
0,QKeras,0.000012712740,None,None,0.999994642597


Save metrics values to file for future reference.

In [27]:
metrics_filename = PREFIX + '/metrics.csv'

# You can disable the writing if necessary.
write_metrics_csv = True

In [28]:
from utils.metrics import write_metrics
if write_metrics_csv:
    write_metrics(metrics_filename, 'QKeras', model_id_qkeras, qmse, qmsle, qmape, xgate_fidelity_qkeras)

In [29]:
from utils.metrics import print_metrics
print_metrics(metrics_filename)

,Date,Framework,ID,MSE,MSLE,MAPE,Fidelity
0,19/04/2024 04:44:50,Keras,smallMLP_1x4x5,NaN,NaN,NaN,NaN
1,20/04/2024 15:13:01,Keras,smallMLP_1x4x5,0.000013080302,NaN,NaN,NaN
2,20/04/2024 15:24:33,Keras,smallMLP_1x4x5,0.000013080302,NaN,NaN,0.999994319741
3,20/04/2024 15:37:11,QKeras,smallMLP_1x4x5,0.000012712740,NaN,NaN,0.999994642597
4,20/04/2024 15:41:56,QKeras,smallMLP_1x4x5,0.000012712740,NaN,NaN,0.999994642597
